# Make median-normalized, false-colour RGB images of the NGVS-VERTICO galaxies

Isaac Cheng - November 2021

Convert i-band, g-band, and u-band data to red, green, and blue channels, respectively. Note that we ignore the flag maps for the RGB images.


In [1]:
import re
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
# 
# Load my own packages
# 
import sys
sys.path.append("/arc/home/IsaacCheng/coop_f2021/packages")
import fits_utils as fu
import plot_utils as pu
# # Reload my packages
# import importlib
# importlib.reload(fu)
# importlib.reload(pu)
GALDIST = 16.5 * u.Mpc  # distance to centre of Virgo cluster. Assume 10% uncertainty

In [2]:
GALAXIES = [
    "IC3392",
    "NGC4189",  # ignored flag map, but bad results
    "NGC4192",
    "NGC4216",  # has unflagged foreground star
    "NGC4222",
    "NGC4254",
    "NGC4294",
    "NGC4298",
    "NGC4299",
    "NGC4302",
    "NGC4321",
    "NGC4330",
    "NGC4351",
    "NGC4380",
    "NGC4383",
    "NGC4388",
    "NGC4396",
    "NGC4402",
    "NGC4405",
    "NGC4419",
    "NGC4424",
    "NGC4450",
    "NGC4501",
    "NGC4522",
    "NGC4532",
    "NGC4535",
    "NGC4548",
    "NGC4567",
    "NGC4568",
    "NGC4569",
    "NGC4579",
    "NGC4580",
    "NGC4606",  # flag map too intrusive
    "NGC4607",
    "NGC4651",
    "NGC4654",
    "NGC4689",
    "NGC4694",
]

INCLINATIONS = [
    68,
    42,  # ignored flag map, but bad results
    83,
    90,  # has unflagged foreground star
    90,
    39,
    74,
    52,
    14,
    90,
    32,
    90,
    48,
    61,
    56,
    83,
    83,
    80,
    46,
    74,
    61,
    51,
    65,
    82,
    64,
    48,
    37,
    49,
    70,
    69,
    40,
    46,
    69,  # flag map too intrusive
    90,
    53,
    61,
    38,
    62,
]  # degrees

for galaxy, i in zip(GALAXIES, INCLINATIONS):
    print("Making", galaxy, "RGB image")
    galpath = f"/arc/home/IsaacCheng/coop_f2021/galaxies/{galaxy}/"
    # 2 arcsec pixel CO data (purely to cutout optical data to match CO data)
    if galaxy == "NGC4321":  # no 9 arcsec data since native is 10 arcsec
        Ico_path = f"/arc/home/IsaacCheng/coop_f2021/vertico_data/v1.3.1/native/{galaxy}/{galaxy}_7m+tp_co21_pbcorr_round_mom0_Kkms-1.fits"
    else:
        Ico_path = f"/arc/home/IsaacCheng/coop_f2021/vertico_data/v1.3/9arcsec/{galaxy}/{galaxy}_7m+tp_co21_pbcorr_9as_round_mom0_Kkms-1.fits"
    Ico_data, co_header = fits.getdata(Ico_path, header=True)
    co_wcs = WCS(co_header)
    # "red" channel (i-band)
    iband_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_i_data.fits"
    # "green" channel (g-band)
    gband_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_g_data.fits"
    # "blue" channel (u-band)
    uband_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_u_data.fits"
    iband_data_uncut, iband_header_uncut = fits.getdata(iband_path, header=True)
    gband_data_uncut, gband_header_uncut = fits.getdata(gband_path, header=True)
    uband_data_uncut, uband_header_uncut = fits.getdata(uband_path, header=True)
    # 
    # Cutout optical data to extent of CO data (optional)
    # 
    iband_data, iband_wcs = fu.cutout_to_target(
        iband_data_uncut, WCS(iband_header_uncut), Ico_data, co_wcs
    )
    gband_data, gband_wcs = fu.cutout_to_target(
        gband_data_uncut, WCS(gband_header_uncut), Ico_data, co_wcs
    )
    uband_data, uband_wcs = fu.cutout_to_target(
        uband_data_uncut, WCS(uband_header_uncut), Ico_data, co_wcs
    )
    # 
    # Median-normalize data on a logarithmic scale and make RGB array
    # 
    rgb_data = pu.lognorm_median(iband_data, gband_data, uband_data, a=1000, norm_factor=1000)
    # 
    # Plot
    # 
    fig, ax = plt.subplots(subplot_kw={"projection": uband_wcs})
    ax.imshow(rgb_data, interpolation="none")
    ax.tick_params(color="w")
    pu.add_scalebar(ax, uband_wcs, dist=GALDIST, color="w")
    ax.set_xlabel("RA (J2000)")
    ax.set_ylabel("Dec (J2000)")
    ax.grid(False)
    # 
    # Add name of galaxy
    # 
    high_i_str = "*" if i >= 80 else ""
    ax.text(
        0.9,
        0.9,
        re.sub(r"(\d+)", " \\1", galaxy) + high_i_str,
        c="w",
        ha="right",
        transform=ax.transAxes,
    )
    fig.savefig(galpath + f"{galaxy}_rgb.pdf")
    plt.close()
print("Done!")

Making IC3392 RGB image


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Making NGC4189 RGB image


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Making NGC4192 RGB image


...

Making NGC4689 RGB image


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Making NGC4694 RGB image


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Done!


# (Miscellaneous) Getting the inclinations of NGVS galaxies


In [ ]:
galaxies = [
    "IC3392", "NGC4064", "NGC4189", "NGC4192", "NGC4216", "NGC4222", "NGC4254",
    "NGC4293", "NGC4294", "NGC4298", "NGC4299", "NGC4302", "NGC4321", "NGC4330",
    "NGC4351", "NGC4380", "NGC4383", "NGC4388", "NGC4394", "NGC4396", "NGC4402",
    "NGC4405", "NGC4419", "NGC4424", "NGC4450", "NGC4457", "NGC4501", "NGC4522",
    "NGC4532", "NGC4533", "NGC4535", "NGC4536", "NGC4548", "NGC4561", "NGC4567",
    "NGC4568", "NGC4569", "NGC4579", "NGC4580", "NGC4606", "NGC4607", "NGC4651",
    "NGC4654", "NGC4689", "NGC4694", "NGC4698", "NGC4713", "NGC4772", "NGC4808",
]  # 49 galaxies

inclinations = [
    68, 70, 42, 83, 90, 90, 39, 67, 74, 52, 14, 90, 32, 90, 48, 61, 56, 83, 32, 83, 80,
    46, 74, 61, 51, 37, 65, 82, 64, 80, 48, 74, 37, 28, 49, 70, 69, 40, 46, 69, 90, 53,
    61, 38, 62, 66, 45, 60, 72
]  # degrees

non_NGVS_galaxies = [
    "NGC4064", "NGC4293", "NGC4394", "NGC4457", "NGC4533", "NGC4536",
    "NGC4561", "NGC4698", "NGC4713", "NGC4772", "NGC4808", # "NGC4606"
]

for galaxy, i in zip(galaxies, inclinations):
    if galaxy in non_NGVS_galaxies:
        continue
    print(f"{i},", end=" ")
    # print(f"{i},")